In [16]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Declare the File Path
file_path = 'C:/Users/ajaym/Desktop/Utility/Project/Modified_Book_for_retail_with_Harry_Potter (2).xlsx'

# Load the Excel file
df = pd.read_excel(file_path)

# Trim column names to remove extra spaces
df.columns = df.columns.str.strip()

# Debug: Print available column names
print("Columns in dataset:", df.columns)

# Ensure required columns exist
required_columns = {'Book_ID', 'Genre', 'Average_Rating', 'Book_Name'}
missing_columns = required_columns - set(df.columns)

if missing_columns:
    print(f"Missing columns in dataset: {missing_columns}")
    exit()  # Stop execution if columns are missing

# One-hot encoding for genres
genre_dummies = pd.get_dummies(df['Genre'], prefix='Genre')

# Feature matrix for KNN
X = pd.concat([df[['Book_ID', 'Average_Rating']], genre_dummies], axis=1)

# Function to recommend books
def recommend_books_by_genre_ml(genre, num_recommendations=5):
    # Filter books belonging to the selected genre
    genre_books = df[df['Genre'] == genre]

    if genre_books.empty:
        print(f"No books found for genre '{genre}'.")
        return pd.DataFrame(columns=['Book_ID', 'Book_Name', 'Average_Rating'])

    # Match feature set with books in the same genre
    genre_books_features = X[X['Book_ID'].isin(genre_books['Book_ID'])].drop(columns=['Book_ID'])

    # Train KNN model
    knn = NearestNeighbors(n_neighbors=num_recommendations, algorithm='auto')
    knn.fit(X.drop(columns=['Book_ID']))

    # Find nearest books
    distances, indices = knn.kneighbors(genre_books_features)

    recommended_books = []
    for idx_list in indices:
        for idx in idx_list:
            book = df.iloc[idx][['Book_ID', 'Book_Name', 'Average_Rating']]
            recommended_books.append(book)

    # Convert to DataFrame and remove duplicates
    recommended_books_df = pd.DataFrame(recommended_books).drop_duplicates().sort_values(by='Average_Rating', ascending=False)
    
    return recommended_books_df.head(num_recommendations)

# User input for genre
print("Choose from : Fantasy || Romance || Fiction || Mystery : \n")
genre = input("Enter the genre you're interested in: ")
recommendations = recommend_books_by_genre_ml(genre=genre, num_recommendations=5)
# Display results
print(f"Recommended Books in '{genre}' genre:")
print(recommendations)


Columns in dataset: Index(['Index', 'Book_ID', 'Genre', 'Average_Rating', 'Book_Name'], dtype='object')
Choose from : Fantasy || Romance || Fiction || Mystery : 



Enter the genre you're interested in:  Fantasy


Recommended Books in 'Fantasy' genre:
    Book_ID           Book_Name  Average_Rating
47      148  Kingdoms of Wonder            4.91
13      114  Kingdoms of Wonder            4.77
41      142  Kingdoms of Wonder            4.28
6       107  Kingdoms of Wonder            4.10
49      150       Dragon's Path            3.96
